# Example for generating photoionization models

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyCloudy as pc
pc.log_.level = 3


from sys import path
path.insert(0,'../CloudyGalaxy/')

from gas_stats import GasStatsBy17 as GasStats
from abundances import ElementalAbundances, convert_xi_to_F
from read_write_cloudy import StellarSpectrum, format_emission_table, make_input_file, make_emission_line_files

### Setting the data paths and cores to use for Cloudy

In [2]:
n_proc              = 6

output_dir          = '/Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/'
model_name          = 'test_model_low_res_jenkins_depletion'
data_path           = '../CloudyGalaxy/input_data/'
star_table_filename = 'Chabrier_constant_SFH_IMF_lower_0p08_IMF_upper_120_star_table.ASCII'
emission_line_list  = format_emission_table('../CloudyGalaxy/input_data/LineList_HII.dat')


### Defining the grid of free parameters for which to produce models

In [3]:
nlogZs = 3
nlogUs = 2
nxis   = 2
ntaus  = 2

logZs = np.linspace(-1.0, 0.7, nlogZs)
logUs = np.linspace(-4.0, -1.0, nlogUs)
xis   = np.linspace(0.1, 0.6, nxis)
logtaus  = np.linspace(-2.0, 0.6, ntaus)
Fs    = np.ones((nlogZs,nxis))*np.nan


### Making the Cloudy input files

In [4]:
pc.print_make_file(dir_ = output_dir)
for i in range(nlogZs):
    logZ = logZs[i]
    xi_to_F = convert_xi_to_F(logZ)
    for j in range(nlogUs):
        for k in range(nxis):
            logU = logUs[j]
            xi = xis[k]
            F = xi_to_F(xi)
            Fs[i,k] = F
            stellar_spectrum = StellarSpectrum(data_path, star_table_filename)
            gas_stats = GasStats(stellar_spectrum, logZ, logU)
            gas_stats.calc_all_parameters()
            elemental_abundances = ElementalAbundances(logZ, F, scaling_method='jenkins09')
            elemental_abundances.calc_all_parameters()
            make_input_file(output_dir, model_name, logZ, logU, xi, emission_line_list, stellar_spectrum, gas_stats, elemental_abundances)

     print_make_file: make_file_printed with cloudy.exe = /Users/dirk/Documents/PhD/scripts/packages/c17.02/source/cloudy.exe
     CloudyInput: Input writen in /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.000e+00_-4.000e+00_1.000e-01.in
     CloudyInput: Input writen in /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.000e+00_-4.000e+00_6.000e-01.in
     CloudyInput: Input writen in /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.000e+00_-1.000e+00_1.000e-01.in
     CloudyInput: Input writen in /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.000e+00_-1.000e+00_6.000e-01.in
     CloudyInput: Input writen in /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.500e-01_-4.000e+00_1.000e-01.in
    

### Running Cloudy

In [5]:
pc.run_cloudy(dir_ = output_dir, n_proc = n_proc, model_name = model_name, use_make = True)

     run_cloudy: running: make -j 6 name="test_model_low_res_jenkins_depletion"
     run_cloudy: ending: make -j 6 name="test_model_low_res_jenkins_depletion"


### Extracting the emission line luminosities from the Cloudy data files and storing them in .npy files

In [6]:
make_emission_line_files(output_dir, model_name, logZs, logUs, xis, logtaus, Fs)

/Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.000e+00_-4.000e+00_1.000e-01
     CloudyModel /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.000e+00_-4.000e+00_1.000e-01: Creating CloudyModel for /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.000e+00_-4.000e+00_1.000e-01
     CloudyModel /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.000e+00_-4.000e+00_1.000e-01: /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.000e+00_-4.000e+00_1.000e-01.rad read
     CloudyModel /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/examples/output_data/test_model_low_res_jenkins_depletion_-1.000e+00_-4.000e+00_1.000e-01: Number of zones: 82
     CloudyModel /Users/dirk/Documents/PhD/scripts/CloudyGalaxy/exampl

ValueError: could not broadcast input array from shape (73,) into shape (146,)